In [1]:
import os
import pandas as pd
base_dir = './kdef/KDEF/'
path = []
emotion = []
angles = []
index = []

for i, j, k in os.walk(base_dir):
#     print(k)
    for img in k:
        name = img.split('.')[0]
        No = name[:4]
        expression = name[4:6]
        angle = name[6:]
        swtich_expression = {
            "AF": 2,
            "AN": 0,
            "DI": 1,
            "HA": 3,
            "NE": 6,
            "SA": 4,
            "SU": 5
        }
        switch_angle = {
            "FL": 0,
            "FR": 1,
            "HL": 2,
            "HR": 3,
            "S" : 4
        }
        
        path.append('kdef/KDEF/'+No + '/' + img) 
        index.append(No)
        emotion.append(swtich_expression[expression])
        angles.append(switch_angle[angle])
       
        
dict = {
        "index" : index,
        "path" : path,
        "angle" : angles,
        "emotion" : emotion,
        
    }
data = pd.DataFrame(dict)
print(data)

     index                         path  angle  emotion
0     BF07   kdef/KDEF/BF07/BF07SAS.JPG      4        4
1     BF07  kdef/KDEF/BF07/BF07SAFL.JPG      0        4
2     BF07  kdef/KDEF/BF07/BF07NEHR.JPG      3        6
3     BF07  kdef/KDEF/BF07/BF07SAHL.JPG      2        4
4     BF07  kdef/KDEF/BF07/BF07DIHR.JPG      3        1
...    ...                          ...    ...      ...
4893  AM10  kdef/KDEF/AM10/AM10HAHR.JPG      3        3
4894  AM10  kdef/KDEF/AM10/AM10SAHR.JPG      3        4
4895  AM10  kdef/KDEF/AM10/AM10SAFL.JPG      0        4
4896  AM10  kdef/KDEF/AM10/AM10NEFR.JPG      1        6
4897  AM10   kdef/KDEF/AM10/AM10HAS.JPG      4        3

[4898 rows x 4 columns]


In [2]:
import dlib
import cv2
import numpy as np
from skimage import io

predictor_path = "shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

neutral = data[(data['emotion'] == 6) &  (data['angle'] == 4)]

others = data[ (data['angle'] == 4)]

print(others)

print(neutral.iloc[1,1])
dict ={}

for i in range(68):
    dict['F'+str(i)]=[]
    
    
    
dict['index']=[]
for i,row in neutral.iterrows():
    img = cv2.imread(row['path'])

    dets = detector(img)
    dict['index'].append(row['index'])
    if len(dets):
         shape = predictor(img, dets[0])
         
         for b in range(68):
            x = shape.part(b).x
            y = shape.part(b).y
            # print(shape.part(b))
            dict['F'+str(b)].append(np.array([x,y]))
                
                # cv2.circle(img=img, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)

            # cv2.imshow(winname="Face", mat=img)
            # cv2.waitKey(1000)
            # # To close the window after the required kill value was provided
            # cv2.destroyAllWindows()




DF2 = pd.DataFrame()

for i,row in others.iterrows():
    dict2 = {}
    print(i)
 
    img = cv2.imread(row['path'])

    dets = detector(img)
    dict2['index']=(row['index'])
    dict2['path'] = row['path']
    if len(dets):
         shape = predictor(img, dets[0])
         t1=[]
         for b in range(68):
            x = shape.part(b).x
            y = shape.part(b).y
            t1.append(np.array([x,y]))
            dict2['F'+str(b)]=np.array([x,y])
         DF2=DF2.append(dict2,ignore_index=True)





            

   

     index                        path  angle  emotion
0     BF07  kdef/KDEF/BF07/BF07SAS.JPG      4        4
12    BF07  kdef/KDEF/BF07/BF07NES.JPG      4        6
13    BF07  kdef/KDEF/BF07/BF07AFS.JPG      4        2
17    BF07  kdef/KDEF/BF07/BF07SUS.JPG      4        5
19    BF07  kdef/KDEF/BF07/BF07ANS.JPG      4        0
...    ...                         ...    ...      ...
4882  AM10  kdef/KDEF/AM10/AM10DIS.JPG      4        1
4886  AM10  kdef/KDEF/AM10/AM10AFS.JPG      4        2
4887  AM10  kdef/KDEF/AM10/AM10SAS.JPG      4        4
4888  AM10  kdef/KDEF/AM10/AM10SUS.JPG      4        5
4897  AM10  kdef/KDEF/AM10/AM10HAS.JPG      4        3

[980 rows x 4 columns]
kdef/KDEF/BM11/BM11NES.JPG
0
12
13
17
19
30
32
36
39
44
55
57
62
69
78
81
89
95
97
103
104
107
111
114
116
119
124
132
142
156
162
166
170
172
173
175
177
181
192
203
204
206
216
218
230
235
237
239
241
261
262
268
270
272
276
277
286
287
301
304
307
308
312
319
324
327
328
340
344
347
354
357
362
368
375
378
382
3

In [43]:
x=[0 for i in range(68)]
y=[0 for i in range(68)]
DF2

0      [161, 493]
1      [151, 500]
2      [146, 519]
3      [148, 504]
4      [150, 502]
          ...    
975    [153, 503]
976    [153, 506]
977    [149, 508]
978    [148, 506]
979    [146, 501]
Name: F3, Length: 980, dtype: object

In [8]:


new = pd.merge(others,DF2, how='inner',on='path')
new1 = new.drop_duplicates(subset="path")
print(new1)
new1.to_csv("new.csv")

    index_x                        path  angle  emotion          F0  \
0      BF07  kdef/KDEF/BF07/BF07SAS.JPG      4        4  [150, 378]   
1      BF07  kdef/KDEF/BF07/BF07NES.JPG      4        6  [143, 387]   
2      BF07  kdef/KDEF/BF07/BF07AFS.JPG      4        2  [132, 405]   
3      BF07  kdef/KDEF/BF07/BF07SUS.JPG      4        5  [137, 383]   
4      BF07  kdef/KDEF/BF07/BF07ANS.JPG      4        0  [133, 378]   
..      ...                         ...    ...      ...         ...   
975    AM10  kdef/KDEF/AM10/AM10DIS.JPG      4        1  [134, 379]   
976    AM10  kdef/KDEF/AM10/AM10AFS.JPG      4        2  [135, 376]   
977    AM10  kdef/KDEF/AM10/AM10SAS.JPG      4        4  [132, 384]   
978    AM10  kdef/KDEF/AM10/AM10SUS.JPG      4        5  [130, 377]   
979    AM10  kdef/KDEF/AM10/AM10HAS.JPG      4        3  [131, 374]   

             F1         F10         F11         F12         F13  ...  \
0    [150, 416]  [365, 601]  [395, 571]  [415, 534]  [426, 494]  ...   
1  

In [9]:
# neutral.join(dict)
import pandas as pd

ndf = pd.DataFrame(dict)
out = pd.merge(neutral,ndf, how='inner')
out1 = out.drop_duplicates(subset='index')
print(out1)
out1.to_csv("out.csv")

    index                        path  angle  emotion          F0          F1  \
0    BF07  kdef/KDEF/BF07/BF07NES.JPG      4        6  [143, 387]  [142, 424]   
1    BM11  kdef/KDEF/BM11/BM11NES.JPG      4        6  [127, 414]  [133, 453]   
2    BM27  kdef/KDEF/BM27/BM27NES.JPG      4        6  [123, 386]  [125, 428]   
3    BF29  kdef/KDEF/BF29/BF29NES.JPG      4        6  [107, 390]  [110, 433]   
4    BM33  kdef/KDEF/BM33/BM33NES.JPG      4        6  [122, 407]  [126, 448]   
..    ...                         ...    ...      ...         ...         ...   
135  BF26  kdef/KDEF/BF26/BF26NES.JPG      4        6  [122, 386]  [124, 428]   
136  BM04  kdef/KDEF/BM04/BM04NES.JPG      4        6  [127, 391]  [130, 433]   
137  AF23  kdef/KDEF/AF23/AF23NES.JPG      4        6  [115, 390]  [120, 432]   
138  BF10  kdef/KDEF/BF10/BF10NES.JPG      4        6  [113, 402]  [118, 444]   
139  AM10  kdef/KDEF/AM10/AM10NES.JPG      4        6  [128, 380]  [132, 422]   

             F2          F3

In [24]:
import pandas as pd
import numpy as np
df_n = pd.read_csv("out.csv")
df_o = pd.read_csv("new.csv")
dict = {1:[55,49],
        2:[41,39],
        3:[48,44],
        4:[43,25],
        5:[58,52],
        6:[40,37],
        7:[46,43],
        8:[40,20],
        9:[23,22]
}
dataset = df_o.iloc[:,1:4]
emotion = df_o.iloc[:,4]


# dataset['id'] = range(len(dataset))
# print(dataset)

# def distance(a,b):
    
#     a = a[1:-1]
#     b = b[1:-1]
    
#     a = a.split(' ')
#     b = b.split(' ')
#     print(a)
#     p1 = np.array([int(a[0]),int(a[1])])
#     p2 = np.array([int(b[0]),int(b[1])])
#     return np.linalg.norm(p1-p2)

def dist(a,b):
    
    a = a.to_numpy()[0]
    b = b.to_numpy()[0]
 
    a = a[1:-1]
    b = b[1:-1]
    a = a.split(' ')
    b = b.split(' ')
    p1 = np.array([int(a[0]),int(a[1])])
    p2 = np.array([int(b[0]),int(b[1])])
    return np.linalg.norm(p1-p2)
    
# # print(df_o)
# temp = []
# for i,row in df_o.iterrows():
#     #  print(i,row['index'])
#     #  print(row[dict[j+1][0]-4],row[dict[j+1][1]-4])
#      t=[]
#      for j in range(9):
#          print((row[dict[j+1][0]-4],row[dict[j+1][1]-4]))
#          print(df_n[df_n['index']==row['index']].iloc[:,dict[j+1][0]-4],df_n[df_n['index']==row['index']].iloc[:,dict[j+1][1]-4])
#          dis_o = distance(row[dict[j+1][0]-4],row[dict[j+1][1]-4])
#          dis_n = dist(df_n[df_n['index']==row['index']].iloc[:,dict[j+1][0]-4],df_n[df_n['index']==row['index']].iloc[:,dict[j+1][1]-4])
         
#          t.append( dis_o - dis_n )
#      temp.append(t)
# tt = pd.DataFrame(temp)
# print(tt)
# tt.to_csv("dataset.csv")




    index                        path  angle
0    BF07  kdef/KDEF/BF07/BF07SAS.JPG      4
1    BF07  kdef/KDEF/BF07/BF07NES.JPG      4
2    BF07  kdef/KDEF/BF07/BF07AFS.JPG      4
3    BF07  kdef/KDEF/BF07/BF07SUS.JPG      4
4    BF07  kdef/KDEF/BF07/BF07ANS.JPG      4
..    ...                         ...    ...
975  AM10  kdef/KDEF/AM10/AM10DIS.JPG      4
976  AM10  kdef/KDEF/AM10/AM10AFS.JPG      4
977  AM10  kdef/KDEF/AM10/AM10SAS.JPG      4
978  AM10  kdef/KDEF/AM10/AM10SUS.JPG      4
979  AM10  kdef/KDEF/AM10/AM10HAS.JPG      4

[980 rows x 3 columns]


In [10]:
print(emotion)

0      5
1      4
2      0
3      6
4      1
      ..
975    2
976    0
977    5
978    6
979    3
Name: emotion, Length: 980, dtype: int64


In [1]:


data = pd.DataFrame(tt)
# emotion = emotion.to_numpy()


from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score

scalar = StandardScaler()

X = scalar.fit_transform(data)
print(X)

encoder = LabelEncoder()
encoder.fit(emotion)
y = encoder.transform(emotion)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(y)

print(dummy_y)


X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.33, random_state=42)   

def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(64, activation='relu'))
	model.add(Dense(128, activation='relu'))
	
	model.add(Dense(7, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=5, verbose=1)
print(estimator)
kfold = KFold(n_splits=5, shuffle=True)
print(kfold)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print(results)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

y_pred=estimator.predict(X_test)

# # model = Sequential()
# # model.add(Dense(64, activation='relu'))
# # model.add(Dense(1, activation='softmax'))

# # model.compile(loss='hinge',
# #               optimizer='adam',
# #               metrics=['accuracy'])
# # model.fit(X_train, y_train)

# # y_pred = model.predict(X_test)

# # print(y_pred)

# # print(metrics.accuracy_score(y_test,y_pred))

# model = KNeighborsClassifier(n_neighbors=6)

# model.fit(X_train,y_train)

# y_pred = model.predict(X_train)

# print(y_pred)

# print(metrics.accuracy_score(y_train,y_pred))

# con = confusion_matrix(y_train,y_pred,[0,1,2,3,5,6])

# print(con)

# model = baseline_model()

# model.fit(X,dummy_y)

# print(model.predict(X))




NameError: name 'pd' is not defined

In [57]:
estimator.fit(X_train,y_train)
y_pred=estimator.predict_proba(X_test)
print(y_pred)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

132/132 [==============================] - 0s 583us/step - loss: 3.1363 - accuracy: 0.3262
Epoch 2/50
132/132 [==============================] - 0s 464us/step - loss: 1.7659 - accuracy: 0.4162
Epoch 3/50
132/132 [==============================] - 0s 491us/step - loss: 1.6788 - accuracy: 0.4421
Epoch 4/50
132/132 [==============================] - 0s 485us/step - loss: 1.6477 - accuracy: 0.4375
Epoch 5/50
132/132 [==============================] - 0s 493us/step - loss: 1.5426 - accuracy: 0.434

In [71]:
y_pred = estimator.predict(X_test)
print(y_test.argmax(axis=1))
print(y_pred)

print(metrics.accuracy_score(y_test.argmax(axis=1),y_pred))
# matrix = metrics.confusion_matrix(y_test, y_pred.argmax())



65/65 [==============================] - 0s 301us/step
[1 5 3 1 6 1 2 0 5 4 0 5 5 3 6 2 1 2 4 6 1 1 0 4 5 6 2 1 3 3 2 5 5 6 2 3 4
 1 3 5 5 0 5 6 0 1 0 5 6 6 0 5 0 4 2 5 1 1 0 2 6 6 6 3 1 1 6 0 6 4 3 6 5 2
 4 2 0 3 0 3 3 6 0 3 2 6 0 6 6 3 5 2 2 1 0 0 1 3 0 0 2 3 4 1 3 3 2 6 2 2 4
 3 4 1 0 5 1 4 0 4 1 0 3 2 6 1 5 3 5 6 4 1 0 4 6 2 5 0 4 2 0 1 2 4 5 6 6 1
 0 3 6 4 2 6 6 5 5 5 4 1 1 3 6 4 2 1 0 4 4 0 3 2 4 1 3 2 0 1 3 6 5 1 1 2 3
 4 5 3 0 6 3 2 1 4 2 2 2 4 5 3 0 4 1 3 1 5 1 5 6 5 2 6 4 5 0 3 5 3 4 4 5 0
 0 0 4 5 6 2 0 2 1 3 4 6 6 1 2 0 6 5 0 2 3 2 1 2 5 5 2 4 4 5 0 4 3 0 2 4 1
 0 2 6 2 1 3 1 6 3 0 3 1 0 0 5 2 1 6 2 4 3 4 0 2 6 6 3 5 4 0 0 2 0 3 3 1 4
 2 1 3 5 6 6 3 1 5 2 0 1 1 0 6 6 5 5 6 6 5 0 4 2 5 4 3 0]
[1 4 3 1 6 1 1 0 0 4 0 0 0 3 6 2 1 3 0 6 0 1 3 0 5 6 2 1 3 6 5 0 0 6 6 3 4
 1 3 1 4 4 1 6 3 1 0 3 6 4 0 4 3 4 2 4 1 2 5 4 5 0 6 3 1 1 4 0 6 5 3 6 2 2
 4 3 6 3 6 3 3 6 4 3 0 6 0 6 6 3 4 2 1 1 2 0 1 3 4 3 2 3 6 0 3 3 2 6 2 2 0
 3 4 1 6 5 1 4 0 2 1 6 3 1 4 1 4 3 3 6 4 0 2 4 6 2 2 0 4 2 5 3